# Groundedness Checking Plugins

A well-known problem with large language models (LLMs) is that they make things up. These are sometimes called 'hallucinations' but a safer (and less anthropomorphic) term is 'ungrounded addition' - something in the text which cannot be firmly established. When attempting to establish whether or not something in an LLM response is 'true' we can either check for it in the supplied prompt (this is called 'narrow grounding') or use our general knowledge ('broad grounding'). Note that narrow grounding can lead to things being classified as 'true, but ungrounded.' For example "I live in Switzerland" is **not** _narrowly_ grounded in "I live in Geneva" even though it must be true (it **is** _broadly_ grounded).

In this notebook we run a simple grounding pipeline, to see if a summary text has any ungrounded additions as compared to the original, and use this information to improve the summary text. This can be done in three stages:

1. Make a list of the entities in the summary text
1. Check to see if these entities appear in the original (grounding) text
1. Remove the ungrounded entities from the summary text

What is an 'entity' in this context? In its simplest form, it's a named object such as a person or place (so 'Dean' or 'Seattle'). However, the idea could be a _claim_ which relates concepts (such as 'Dean lives near Seattle'). In this notebook, we will keep to the simpler case of named objects.

Let us first define our grounding text:


In [1]:
grounding_text = """I am by birth a Genevese, and my family is one of the most distinguished of that republic.
My ancestors had been for many years counsellors and syndics, and my father had filled several public situations
with honour and reputation. He was respected by all who knew him for his integrity and indefatigable attention
to public business. He passed his younger days perpetually occupied by the affairs of his country; a variety
of circumstances had prevented his marrying early, nor was it until the decline of life that he became a husband
and the father of a family.

As the circumstances of his marriage illustrate his character, I cannot refrain from relating them. One of his
most intimate friends was a merchant who, from a flourishing state, fell, through numerous mischances, into poverty.
This man, whose name was Beaufort, was of a proud and unbending disposition and could not bear to live in poverty
and oblivion in the same country where he had formerly been distinguished for his rank and magnificence. Having
paid his debts, therefore, in the most honourable manner, he retreated with his daughter to the town of Lucerne,
where he lived unknown and in wretchedness. My father loved Beaufort with the truest friendship and was deeply
grieved by his retreat in these unfortunate circumstances. He bitterly deplored the false pride which led his friend
to a conduct so little worthy of the affection that united them. He lost no time in endeavouring to seek him out,
with the hope of persuading him to begin the world again through his credit and assistance.

Beaufort had taken effectual measures to conceal himself, and it was ten months before my father discovered his
abode. Overjoyed at this discovery, he hastened to the house, which was situated in a mean street near the Reuss.
But when he entered, misery and despair alone welcomed him. Beaufort had saved but a very small sum of money from
the wreck of his fortunes, but it was sufficient to provide him with sustenance for some months, and in the meantime
he hoped to procure some respectable employment in a merchant's house. The interval was, consequently, spent in
inaction; his grief only became more deep and rankling when he had leisure for reflection, and at length it took
so fast hold of his mind that at the end of three months he lay on a bed of sickness, incapable of any exertion.

His daughter attended him with the greatest tenderness, but she saw with despair that their little fund was
rapidly decreasing and that there was no other prospect of support. But Caroline Beaufort possessed a mind of an
uncommon mould, and her courage rose to support her in her adversity. She procured plain work; she plaited straw
and by various means contrived to earn a pittance scarcely sufficient to support life.

Several months passed in this manner. Her father grew worse; her time was more entirely occupied in attending him;
her means of subsistence decreased; and in the tenth month her father died in her arms, leaving her an orphan and
a beggar. This last blow overcame her, and she knelt by Beaufort's coffin weeping bitterly, when my father entered
the chamber. He came like a protecting spirit to the poor girl, who committed herself to his care; and after the
interment of his friend he conducted her to Geneva and placed her under the protection of a relation. Two years
after this event Caroline became his wife."""

## Set up Semantic Kernel

We prepare our kernel in the usual way:


In [ ]:
!python -m pip install semantic-kernel==1.0.5

In [3]:
import sys
import os

# Add the samples directory to the Python path
samples_dir = r'C:\Users\sulma\OneDrive\Documents\GitHub\semantic-kernel\python\samples'
sys.path.append(samples_dir)

try:
    from services import Service
    from service_settings import ServiceSettings

    service_settings = ServiceSettings(
        global_llm_service="AzureOpenAI")  # Example setting

    # Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
    selectedService = (
        Service.AzureOpenAI
        if service_settings.global_llm_service is None
        else Service(service_settings.global_llm_service.lower())
    )
    print(f"Using service type: {selectedService}")
except ModuleNotFoundError as e:
    print(f"Error: {e}")
    print("Ensure that 'service_settings.py' is in the correct directory and properly named.")
    # Additional debugging information
    print("Current Python path:")
    for path in sys.path:
        print(path)

Using service type: Service.AzureOpenAI


In [4]:
from dotenv import load_dotenv
from semantic_kernel import Kernel

kernel = Kernel()


env_path = r'C:\Users\sulma\OneDrive\Documents\GitHub\semantic-kernel\python\samples\getting_started\.env'
load_dotenv(env_path)

service_id = None

# Load the API keys and other settings from the environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
openai_org_id = os.getenv('OPENAI_ORG_ID')
azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
azure_chat_deployment_name = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')
azure_openai_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')


if selectedService == Service.OpenAI:

    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"

    kernel.add_service(


        OpenAIChatCompletion(service_id=service_id,
                             ai_model_id="gpt-3.5-turbo-1106"),


    )


elif selectedService == Service.AzureOpenAI:

    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"

    kernel.add_service(


        AzureChatCompletion(
            service_id=service_id,
            api_key=azure_openai_api_key,           # Use the api_key from the .env file
            # Use the deployment_name from the .env file
            deployment_name=azure_chat_deployment_name,
            endpoint=azure_openai_endpoint,         # Use the endpoint from the .env file
            env_file_path=env_path,




        )
    )

## Import the Plugins

We are going to be using the grounding plugin, to check its quality, and remove ungrounded additions:


In [5]:
# note: using plugins from the samples folder
plugins_directory = "../../../prompt_template_samples/"

groundingSemanticFunctions = kernel.add_plugin(parent_directory=plugins_directory, plugin_name="GroundingPlugin")

We can also extract the individual semantic functions for our use:


In [6]:
entity_extraction = groundingSemanticFunctions["ExtractEntities"]
reference_check = groundingSemanticFunctions["ReferenceCheckEntities"]
entity_excision = groundingSemanticFunctions["ExciseEntities"]

## Calling Individual Semantic Functions

We will start by calling the individual grounding functions in turn, to show their use. For this we need to create a same summary text:


In [7]:
summary_text = """
My father, a respected resident of Milan, was a close friend of a merchant named Beaufort who, after a series of
misfortunes, moved to Zurich in poverty. My father was upset by his friend's troubles and sought him out,
finding him in a mean street. Beaufort had saved a small sum of money, but it was not enough to support him and
his daughter, Mary. Mary procured work to eke out a living, but after ten months her father died, leaving
her a beggar. My father came to her aid and two years later they married when they visited Rome.
"""

summary_text = summary_text.replace("\n", " ").replace("  ", " ")
print(summary_text)

 My father, a respected resident of Milan, was a close friend of a merchant named Beaufort who, after a series of misfortunes, moved to Zurich in poverty. My father was upset by his friend's troubles and sought him out, finding him in a mean street. Beaufort had saved a small sum of money, but it was not enough to support him and his daughter, Mary. Mary procured work to eke out a living, but after ten months her father died, leaving her a beggar. My father came to her aid and two years later they married when they visited Rome. 


Some things to note:

- The implied residence of Geneva has been changed to Milan
- Lucerne has been changed to Zurich
- Caroline has been renamed as Mary
- A reference to Rome has been added

The grounding plugin has three stages:

1. Extract entities from a summary text
2. Perform a reference check against the grounding text
3. Excise any entities which failed the reference check from the summary

Now, let us start calling individual semantic functions.


### Extracting the Entities

The first function we need is entity extraction. We are going to take our summary text, and get a list of entities found within it. For this we use `entity_extraction()`:


In [8]:
extraction_result = await kernel.invoke(
    entity_extraction,
    input=summary_text,
    topic="people and places",
    example_entities="John, Jane, mother, brother, Paris, Rome",
)

print(extraction_result)

<entities>
- father: The speaker's male parent.
- Milan: A city in Italy.
- Beaufort: A merchant who was a friend of the speaker's father and moved to Zurich in poverty.
- Zurich: A city in Switzerland.
- daughter: Refers to Mary, Beaufort's daughter who had to work to support herself and her father.
- Mary: Beaufort's daughter who had to work to support herself and her father and later married the speaker's father in Rome.
- Rome: The capital city of Italy.
</entities>


So we have our list of entities in the summary


### Performing the reference check

We now use the grounding text to see if the entities we found are grounded. We start by adding the grounding text to our context:


With this in place, we can run the reference checking function. This will use both the entity list in the input, and the `reference_context` in the context object itself:


In [9]:
grounding_result = await kernel.invoke(reference_check, input=extraction_result.value, reference_context=grounding_text)

print(grounding_result)

Something went wrong in function invocation. During function invocation: 'GroundingPlugin-ReferenceCheckEntities'. Error description: 'Error occurred while invoking function ReferenceCheckEntities: ("<class 'semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion.AzureChatCompletion'> service failed to complete the prompt", RateLimitError("Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 86400 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}"))'


KernelInvokeException: Error occurred while invoking function: 'GroundingPlugin-ReferenceCheckEntities'

So we now have a list of ungrounded entities (of course, this list may not be well grounded itself). Let us store this in the context:


### Excising the ungrounded entities

Finally we can remove the ungrounded entities from the summary text:


In [10]:
excision_result = await kernel.invoke(entity_excision, input=summary_text, ungrounded_entities=grounding_result.value)

print(excision_result)

NameError: name 'grounding_result' is not defined